In [42]:
# Installation des nouvelles briques
#!pip install polars pyarrow

# Import Librairies

In [43]:
import polars as pl
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin

# Pour conserver les noms de colonnes dans le pipeline
set_config(transform_output="pandas")

# Fonction de Préparation des données

In [44]:
class PolarsPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, skip_first_row=True):
        self.skip_first_row = skip_first_row

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X ici sera une liste de chemins [sirh, eval, sondage]
        sirh_path, eval_path, sondage_path = X
        
        # Votre logique Polars
        df_sirh = pl.read_csv(sirh_path)
        df_eval = pl.read_csv(eval_path)
        df_sondage = pl.read_csv(sondage_path)

        df_eval = df_eval.with_columns(
            pl.col("eval_number").str.replace("E_", "").cast(pl.Int64)
        )

        df_final = (
            df_sirh
            .join(df_eval, left_on="id_employee", right_on="eval_number", how="inner")
            .join(df_sondage, left_on="id_employee", right_on="code_sondage", how="inner")
        )

        df_final = df_final.with_columns([
            (pl.col("augementation_salaire_precedente")
             .str.replace(" %", "")
             .cast(pl.Float64) / 100).alias("augmentation_salaire_precedente_pourcentage")
        ])

        if self.skip_first_row:
            df_final = df_final.tail(-1)

        # On convertit en Pandas à la toute fin du transform pour Scikit-Learn
        return df_final.to_pandas()

In [45]:
# 1. Le préprocesseur Polars (Nettoyage + Jointures)
# 2. Le ColumnTransformer (Standardisation + OneHot)
# 3. Le Classifieur

global_pipeline = Pipeline(steps=[
    ('polars_cleaner', PolarsPreprocessor(skip_first_row=True)),
    ('sklearn_preprocessor', preprocessor), # Votre ColumnTransformer déjà défini
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

# Test

In [46]:

# 1. Définition des fichiers sources
raw_files = ['extrait_sirh.csv', 'extrait_eval.csv', 'extrait_sondage.csv']

# 2. On prépare la cible (y) pour l'entraînement (via la fonction Polars vue précédemment)
# C'est nécessaire car la cible n'est connue qu'après la jointure des fichiers
data_full = prepare_data_polars(*raw_files)
y_train = data_full["target"].to_numpy()

# 3. Entraînement du Pipeline Global
# On passe la liste des chemins en X
global_pipeline.fit(raw_files, y_train)

print("✅ Pipeline entraîné avec succès !")



✅ Pipeline entraîné avec succès !


In [47]:
# 4. Test de prédiction
# On simule l'arrivée de nouveaux fichiers (ici on reprend les mêmes pour le test)
predictions = global_pipeline.predict(raw_files)
probabilities = global_pipeline.predict_proba(raw_files)[:, 1]

# 5. Affichage des premiers résultats
resultats = pl.DataFrame({
    "Prediction": predictions,
    "Probabilité": probabilities
})

print("\n--- Premiers résultats de prédiction ---")
print(resultats.head())


--- Premiers résultats de prédiction ---
shape: (5, 2)
┌────────────┬─────────────┐
│ Prediction ┆ Probabilité │
│ ---        ┆ ---         │
│ i64        ┆ f64         │
╞════════════╪═════════════╡
│ 0          ┆ 0.04        │
│ 1          ┆ 0.8         │
│ 0          ┆ 0.21        │
│ 0          ┆ 0.09        │
│ 0          ┆ 0.05        │
└────────────┴─────────────┘


# Sauvegarder le pipeline complet (Preprocessing + Modèle)

In [48]:
# Sauvegarde d'UN SEUL fichier
joblib.dump(global_pipeline, 'full_techNova_pipeline.pkl')

['full_techNova_pipeline.pkl']

# FastAPI

In [49]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.
